In [ ]:
!pip install langchain==0.0.147
!pip install tiktoken==0.3.2
!pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 626.5/626.5 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 53.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 52.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 33.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 5.8 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.10
    Uninstalling SQLAlchemy-2.0.10:
      Successfully uninstalled SQLAlchemy-2.0.10
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/pub

In [ ]:
import pandas as pd
from langchain.schema import Document
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMChain
import tiktoken
from langchain import ConversationChain
from langchain.memory import ConversationBufferMemory
import sys
sys.path.append('drive/MyDrive/Colab Notebooks')
import pickle
import os
os.environ["OPENAI_API_KEY"]= "YOUR API KEY"

## Load File

In [ ]:
task = '3'
rev_label = False
shuffle = False
file_name = "mbti_test_zero_standard_{}.csv".format(task)

with open('Kaggle/test.pkl', 'rb') as f:
  data = pickle.load(f)
def process_data(data):
  poster = data['posts_text']
  label = data['annotations']
  label_lookup = {'E': 1, 'I': 0, 'S': 1, 'N':0, 'T': 1, 'F': 0, 'J': 1, 'P':0}
  persona_lookup = {}
  poster_data = [{'posts': t, 'label0': label_lookup[list(label[i])[0]],
          'label1': label_lookup[list(label[i])[1]],'label2': label_lookup[list(label[i])[2]],
          'label3': label_lookup[list(label[i])[3]]} for i,t in enumerate(poster)]
  return poster_data
poster_data = process_data(data)
texts = [item['posts'] for item in poster_data]
labels = [item['label{}'.format(task)] for item in poster_data]

In [ ]:
texts[0]

## Define Prompt

In [ ]:
if task == '0':
  if not rev_label:
    trait_choice = 'A: "Extraversion" or B: "Introversion"'
  else:
    trait_choice = 'A: "Introversion" or B: "Extraversion"'
elif task == '1':
  if not rev_label:
    trait_choice = 'A: "Sensing" or B: "Intuition"'
  else:
    trait_choice = 'A: "Intuition" or B: "Sensing"'
elif task == '2':
  if not rev_label:
    trait_choice = 'A: "Thinking" or B: "Feeling"'
  else:
    trait_choice = 'A: "Feeling" or B: "Thinking"'
elif task == '3':
  if not rev_label:
    trait_choice = 'A: "Judging" or B: "Perceiving"'
  else:
    trait_choice = 'A: "Perceiving" or B: "Judging"'

def cteat_agent(trait_choice):
  perfix_temp = 'You are an AI assistant who specializes in text analysis. You will complete a text analysis task. '\
          'The task is as follows: according to a set of posts written by an author, '\
          'predicting whether the author is {}.'.format(trait_choice)
  Prompt_temp = perfix_temp + \
          '\nAUTHOR\'S POSTS: {context}\n' + \
          'The author is {}. Provide a choice in the format: "CHOICE: <A/B>" and do not give the explanation.'.format(trait_choice)
  prompt = PromptTemplate(
      input_variables=["context"],
      template=Prompt_temp)
  llm = OpenAI(temperature=0.0, model_name='gpt-3.5-turbo-0301')
  agent = LLMChain(llm=llm, prompt=prompt, verbose=False)
  return agent

In [ ]:
from sklearn.utils import shuffle

def shuffle_post(posts_list):
  shuffle_index = shuffle([i for i in range(len(posts_list))], random_state=0)
  shuffle_post_list = [posts_list[j] for j in shuffle_index]
  return shuffle_post_list

if not os.path.exists(file_name):
  rest_sample = 0
  head = {"text": [], 'gold': [], 'answers': []}
  data = pd.DataFrame(head)
  data.to_csv(file_name, mode='w', index=False, header=True)
else:
  data = pd.read_csv(file_name)
  rest_sample = len(list(data['gold']))
print('Done: {}/{}'.format(rest_sample, len(texts)))
tokenizer = tiktoken.get_encoding("cl100k_base")
agent = cteat_agent(trait_choice)
for i in range(rest_sample, len(texts)):
  print('Done: {}/{}'.format(i, len(texts)))
  posts = ''
  count = 1
  if shuffle:
    posts_list = shuffle_post(texts[i])
  else:
    posts_list = texts[i]
  for j in range(len(posts_list)):
    if len(posts_list[j])>10:
      post = tokenizer.decode(tokenizer.encode(posts_list[j].replace('{', '').replace('}', ''))[:80])
      posts += 'Post{}: {}; '.format(count,post)
      count += 1
  print(posts)
  answer = agent.predict(context=posts)
  print(answer)
  api_data = pd.DataFrame([[posts, labels[i], answer]])
  api_data.to_csv(file_name, mode='a', index=False, header=False)